# GPT Action Library (ミドルウェア): Azure Function

## はじめに

このページでは、GPT ActionとSpecific applicationを接続するミドルウェアを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報について理解しておくことを確認してください：

- [GPT Actionsの紹介](https://platform.openai.com/docs/actions)
- [GPT Actions Libraryの紹介](https://platform.openai.com/docs/actions/actions-library)
- [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

この特定のGPT Actionは、MSFTのクラウドベースの関数ビルダーである**Azure Function**の構築方法の概要を提供します。このドキュメントは、ユーザーがOAuth保護されたAzure FunctionをGPT Actionに接続し、サンプルアプリケーションに接続するためのセットアップを支援します。

### 価値 + ビジネス活用事例

**価値**: ユーザーは、ChatGPTの自然言語機能を活用してAzure Functionに直接接続できるようになりました。これは以下のような方法で実現できます：

- GPT Actionsの100k文字制限: ユーザーはミドルウェアを使用してAPIからのテキストレスポンスを前処理できます。例えば、ミドルウェアでOpenAIのAPIを使用してテキストを要約してからChatGPTに送り返すことができます。
- 通常、アクションではユーザーはSaaS APIがテキストを返すことに依存しています。ベンダーAPIからのレスポンスを理解しやすいテキストに変換でき、構造化データや非構造化データなど、さまざまなデータタイプを処理できます。
- テキストだけでなくファイルを返すことができます。これは、データ分析用のCSVファイルを表示したり、PDFファイルを取得してChatGPTがアップロードとして扱うようにする場合に便利です。

**使用例**: 
- ユーザーがSharepointでファイルを検索する必要があるが、ChatGPTとSharepointの間にミドルウェアアプリが必要な場合
- ユーザーがAzure functionで複数のステップを連続して構築しており、ChatGPTを使用してそのプロセスを開始できるようにする必要がある場合

## アプリケーション情報

### アプリケーションキーリンク

開始する前に、アプリケーションの以下のリンクを確認してください：
- アプリケーションWebサイト: https://learn.microsoft.com/en-us/azure/azure-functions/
- アプリケーションAPIドキュメント: https://learn.microsoft.com/en-us/azure/azure-functions/functions-reference/

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：
- Azure Function AppsとAzure Entra App Registrationsを作成するアクセス権限を持つAzure Portal

## アプリケーションのセットアップ

### アプリのインストール

Azure Functionsの言語とデプロイオプションについて詳しくは、ドキュメントの左側にある[こちら](https://learn.microsoft.com/en-us/azure/azure-functions/functions-overview?pivots=programming-language-csharp)をご覧ください。

#### オプション1: VSCodeを使用する

VSCodeを使用してデプロイする方法については、Microsoftのドキュメント[こちら](https://learn.microsoft.com/en-us/azure/azure-functions/functions-develop-vs-code?tabs=node-v4,python-v2,isolated-process\&pivots=programming-language-javascript)を参照してください。このアプローチに慣れている場合は、自由にご利用ください。


#### オプション2: Azure Portalで直接実行する

Azure Portalを使用してデプロイする方法については、ドキュメント[こちら](https://learn.microsoft.com/en-us/azure/azure-functions/functions-create-function-app-portal?pivots=programming-language-javascript)を参照してください。ここでは例を使って段階的に説明します。

##### パート1: 関数の作成

![](../../../images/create_function_app.png)

1. [Azure Function app](https://learn.microsoft.com/en-us/azure/azure-functions/functions-overview?pivots=programming-language-csharp)を作成します。私は以下の設定を使用しましたが、あなたが慣れているものを使用できます。すべての言語/オペレーティングシステムがコンソールで直接関数を編集できるわけではないことに注意してください - 私が以下で選択した組み合わせでは可能です。私のウォークスルーでは、すべてをデフォルトのままにして、以下の選択を行いました。以下の設定は、[こちら](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_action_sharepoint_doc)と[こちら](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_action_sharepoint_text)のSharePoint Node.jsソリューションで、そのまま動作します。

   1. 基本設定

      1. _コードまたはコンテナイメージをデプロイしますか？:_ **コード**

      2. _ランタイムスタック:_ **Node.js**

      3. _オペレーティングシステム:_ **Windows**

   2. ネットワーク

      1. _パブリックアクセスを有効にする_: **オン（GPTに接続するために必要）**

2. 上記を完了すると、「デプロイメント」ページに移動します。デプロイメントが完了したら（数分程度で完了するはずです）、**「リソースに移動」**をクリックしてFunction Appに戻ります。

  > 初回試行時にエラーが発生する場合がありますが、再度作成をクリックすると正常に動作する可能性があります。

##### パート2: 認証の設定

3. Azure Function Appの左側メニューで、**Settings**メニュー下の**Authentication**をクリックします。

   1. アイデンティティプロバイダーを追加

   2. アイデンティティプロバイダーとして**Microsoft**を選択します。

   3. テナントタイプとして**Workforce**を選択

   4. **新しいアプリケーションを作成します。** 既存のアプリケーションを使用する場合も手順はほぼ同様ですが、新しいアプリケーションを作成する方が簡単です。「Easy Auth」を使用してコールバックURLとAPIが自動的に公開されるためです。詳細については[**こちら**](https://learn.microsoft.com/en-us/azure/app-service/overview-authentication-authorization)をご覧ください。

   5. このページの他の設定はすべてデフォルトのままにしますが、社内ガイドラインに基づいて自由に変更してください。

   6. **permissions**タブで、**Add Permission**をクリックし、**Files.Read.All**と**Sites.ReadAll**を追加してから**Add**をクリックします。これにより、このアプリケーションがファイルを読み取ることができるようになり、Microsoft Graph Search APIを使用するために重要です。[こちら](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_action_sharepoint_doc)と[こちら](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_action_sharepoint_text)のSharePointソリューションを使用しない場合は、この手順をスキップできます。

4. 作成が完了したら、**作成したエンタープライズアプリケーションをクリックします**（つまり、Function Appページを離れて、作成したエンタープライズアプリケーションのページに移動します）**。** 次に、アプリケーションにログインするユーザーを偽装してAzure Functionを実行するための追加の権限を付与します。詳細については[こちら](https://learn.microsoft.com/en-us/azure/app-service/configure-authentication-provider-aad?tabs=workforce-tenant)をご覧ください。

   1. メインページで「**View API Permissions**」をクリック

   2. **APIs my organization uses**で**Microsoft Azure App Service**を検索し、**user\_impersonation**を見つけます

   3. それを追加し、Azure PortalのAdminに**Grant Admin Consent**を依頼する必要があります。

5) **そのエンタープライズアプリケーション内で**、左側メニューの**Manage**下にある**「Expose an API」**をクリックし、**Copy to Clipboard**ボタンを使用して作成された**scope**をコピーします。スコープは「api://\<insert-uuid>/user\_impersonation」のような形式になります。**後で使用するため** `SCOPE` **として保存してください。**

6) 左側メニューの**Manage**下にある**「Authentication」**をクリック

   1. **Web**セクションの下に、1つのコールバックURIが自動的に追加されていることがわかります。テスト用にPostmanリダイレクトURI（<https://oauth.pstmn.io/v1/callback>）を追加してください。

7) 左側で**Overview**に移動します。**application (client) ID**と**directory (tenant) ID**をコピーし、**後で使用するため** `CLIENT_ID` **と** `TENANT_ID` **として保存してください。**

##### パート3: テスト関数の設定

8. **Function App** のホームに移動してから戻ることで、ページを離れます。

9. **Create Function** をクリックします。この例では、ポータル内で開発しますが、VSCodeや他のIDEを使用することもできます。

   1. **HTTP trigger** を選択します

   2. **Authorization Level** では、任意のキータイプを選択できます。

      1. 初回はエラーが発生する可能性がありますが、Functionは作成されている可能性が高いので、ページを更新して確認してください。

10. 作成したfunctionをクリックします（表示されない場合は、更新をクリックする必要があるかもしれません）。**Get Function URL** をクリックして、Postmanでテストするために保存します。このURLは、後でGPTに組み込む際にOpenAPI仕様を作成するときにも使用します。

![](../../../images/get_function_url.png)

11. 関数アプリに戻り、**Configuration** をクリックします。`MICROSOFT_PROVIDER_AUTHENTICATION_SECRET` 変数の値を表示し、それをコピーして（高度な編集をクリックしてコピー）、**後で使用するために保存します。**

この時点で、テスト関数が作成され、**クライアント ID、テナント ID、シークレット、スコープ、および関数 URL** を保存しているはずです。これで Postman で認証をテストする準備が整いました。

##### パート4: PostmanでAPIの認証をテストする

12. 作成したエンドポイントをPostmanで以下のOAuth設定を使用してテストしてください：

    1. **Grant Type:** Authorization Code

    2. **Auth URL**: https://login.microsoftonline.com/`TENANT_ID`/oauth2/v2.0/authorize

    3. **Auth Token URL**: https://login.microsoftonline.com/`TENANT_ID`/oauth2/v2.0/token

    4. **Client ID:** 上記ステップ7の`CLIENT_ID`

    5. **Client secret:** 上記ステップ11の`MICROSOFT_PROVIDER_AUTHENTICATION_SECRET`

    6. **Scope**: 上記ステップ5の`SCOPE`

    7. **Client credentials**: Send client credentials in body

13. **Get New Access Token**をクリックし、上記ステップ10で保存したエンドポイントにアクセスする必要があります。成功した場合、次のレスポンスが表示されるはずです：`"This HTTP triggered function executed successfully. Pass a name in the query string or in the request body for a personalized response."`

##### パート5: Azure Function上でアプリケーションをセットアップする

これは別途実行する必要があり、あなたのアプリ固有の作業です。その例については[Sharepoint Cookbook]((https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_action_sharepoint_doc))を参照してください。

##### パート6: ChatGPTのセットアップ

14. エンドポイント用のOpenAPI仕様を生成します。

15. それをGPTのActionsセクションに貼り付け、認証タイプとしてOAuthを選択します。上記のPostmanで行ったのと同じ方法でOAuth設定を入力します。

16. アクションを保存すると、GPT設定の下部にコールバックURIが表示されます。そのURLをコピーし、**Azure PortalのFunction Appに戻ります**。

17. **設定**の下の**認証**をクリックし、次にEntraアプリケーションをクリックします。

18. そこに移動したら、**管理**セクションの下の**認証**をクリックします。

19. そのページの**Web**セクションの下に新しいリダイレクトURIを追加し、ステップ16で取得したコールバックURIを貼り付けて、保存をクリックします。

20. GPTをテストすると、期待通りに動作するはずです。

## ChatGPT ステップ

### カスタムGPT指示

*これはアプリケーション固有のものです。例については[Sharepoint Cookbook]((https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_action_sharepoint_doc))を参照してください*

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

以下は、このMiddlewareに接続する際の例です。このセクションでは、あなたのアプリケーションと関数の情報を挿入する必要があります。

In [ ]:
openapi: 3.1.0
info:
  title: {insert title}
  description: {insert description}
  version: 1.0.0
servers:
  - url: https://{your_function_app_name}.azurewebsites.net/api
    description: {insert description}
paths:
  /{your_function_name}?code={enter your specific endpoint id here}:
    post:
      operationId: {insert operationID}
      summary: {insert summary}
      requestBody: 
{the rest of this is specific to your application}

## 認証手順

以下は、このサードパーティアプリケーションでの認証設定に関する手順です。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

### 事前アクション手順

ChatGPTで認証を設定する前に、アプリケーションで以下の手順を実行してください。

*認証の設定については、上記のステップ2と4に従ってください*

### ChatGPTにおいて

ChatGPTで、「Authentication」をクリックし、**「OAuth」**を選択します。以下の情報を入力してください。

- **Client ID**: *上記のステップ12を参照*
- **Client Secret**: *同上*
- **Authorization URL**: *同上*
- **Token URL**: *同上*
- **Scope**: *同上*
- **Token**: *同上*

### アクション後のステップ

ChatGPTで認証を設定したら、アプリケーションで以下の手順に従ってActionを完了してください。

*このアプリケーションのテストについては上記を参照してください*

*私たちに優先的に取り組んでほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。*